## Práctica parte FastAPI

In [1]:
!pip install -q fastapi uvicorn pyngrok scikit-learn transformers joblib nest_asyncio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.2 MB/s eta 0:00:00


### Crear API

#### Configurar ngrok

In [2]:
!ngrok config add-authtoken 2tN08g6yTDm1kjgafyqZfk5BBEp_6fNE8rfjq9sFQKnJM8M9R

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from transformers import pipeline
from pyngrok import ngrok
from typing import Dict
import uvicorn
import nest_asyncio
import threading

# Aplicar nest_asyncio para evitar el error de asyncio.run()
nest_asyncio.apply()

app = FastAPI()

# Base de datos en memoria
models_db  = {
    "sentiment-analysis": {"description": "Modelo para analizar sentimientos"},
    "summarization": {"description": "Modelo para resumir textos"},
}

# Generación de texto
@app.post("/generate_text")
def generate_text(prompt: str):
    text_generator = pipeline("text-generation", model="distilgpt2")
    generated_text = text_generator(prompt, max_length=50, num_return_sequences=1)
    return {"generated_text": generated_text[0]["generated_text"]}

# Obtener lista de modelos disponibles
@app.get("/models")
def get_models():
    """Obtiene la lista de modelos registrados en la API."""
    return {"models": models_db}

# Agregar un nuevo modelo
class ModelCreateRequest(BaseModel):
    description: str

@app.put("/models/{model_name}")
def add_model(model_name: str, model_data: ModelCreateRequest):
    if model_name in models_db:
        raise HTTPException(status_code=400, detail="El modelo ya existe")

    models_db[model_name] = {"description": model_data.description}
    return {"message": f"Modelo '{model_name}' agregado correctamente", "models": models_db}

# Eliminar un modelo
@app.delete("/models/{model_name}")
def delete_model(model_name: str):
    if model_name not in models_db:
        raise HTTPException(status_code=404, detail="El modelo no existe")

    del models_db[model_name]
    return {"message": f"Modelo '{model_name}' eliminado correctamente", "models": models_db}

# Endpoint de resumen de texto usando Transformers
@app.get('/summary')
def summary_generator(query: str):
    summarization_pipeline = pipeline('summarization')
    return summarization_pipeline(query)

# Iniciar el servidor en un puerto específico
def start_api():
    # Abrir el túnel ngrok
    public_url = ngrok.connect(8000).public_url
    print(f"API disponible en: {public_url}")

    # Ejecutar Uvicorn en un hilo
    server_thread = threading.Thread(target=uvicorn.run, args=(app,), kwargs={"host": "0.0.0.0", "port": 8000})
    server_thread.start()

In [4]:
start_api()

API disponible en: https://a43b-34-171-247-22.ngrok-free.app
